In [1]:
import pandas as pd
import numpy as np

/var/folders/vy/kftwr_sx3kjbxvnd1nvc851m0000gn/T/ipykernel_84683/2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
file_path = "../data/raw/kdkf_2016_raw.xlsx"

In [3]:
df = pd.read_excel(file_path, sheet_name="AR_HRM2", header=1)

In [4]:
df.head()

,Unnamed: 0,Unnamed: 1,in 1000 Franken,2015,2016,2016.1,2017
0,ERFOLGSRECHNUNG,NaN,NaN,NaN,NaN,NaN,NaN
1,30,NaN,Personalaufwand,75750.5,74019.7,91571.0,92589.0
2,31,NaN,Sach- und übriger Betriebsaufwand,47208.0,43908.2,49651.0,49081.0
3,davon 314,NaN,baulicher und betrieblicher Unterhalt,16760.7,13491.7,15908.0,13803.0
4,davon 3180,NaN,Wertberichtigungen auf Forderungen,0.0,0.0,0.0,0.0


In [5]:
def rename_columns_based_on_year(
        df: pd.DataFrame, base_year: str
    ) -> pd.DataFrame:
        # Define the base renaming dictionary
        renaming_dict = {
            "Unnamed: 0": "Ref-ID",
            "in 1000 Franken": "Name",
            "en 1000 frs.": "Name",
            f"{base_year}": "Budget y",
            f"{base_year}.1": "Realized",
            str(int(base_year) + 1): "Budget y+1",
        }

        df.columns = df.columns.map(str)

        # Filter out columns not in the renaming dictionary
        df = df[df.columns.intersection(renaming_dict.keys())]

        # Rename the columns
        df = df.rename(columns=renaming_dict)

        return df

In [6]:
df = df.drop(df.index[0])
df = rename_columns_based_on_year(df, 2016)

In [7]:
df

,Ref-ID,Budget y,Realized,Budget y+1
1,30,74019.7,91571.0,92589.0
2,31,43908.2,49651.0,49081.0
3,davon 314,13491.7,15908.0,13803.0
4,davon 3180,0.0,0.0,0.0
5,330,14300.5,13916.0,14406.0
...,...,...,...,...
179,HRM2-Tabelle 18.12,380140.7,387289.0,406743.0
180,HRM2-Tabelle 18.15,361416.0,381633.0,382360.0
181,HRM2-Tabelle 18.11,395454.5,415475.0,414742.0
182,NaN,11929.7,-3966.0,17926.0


In [8]:
hrm1_to_hrm2_dict = {
    "1": (["1"], [1.0]),
    "2": (["2"], [1.0]),
    "3": (["3"], [1.0]),
    "4": (["4"], [1.0]),
    "5": (["5"], [1.0]),
    "6": (["6"], [1.0]),
    "30": (["30"], [1.0]),
    "31": (["31"], [1.0]),
    "33": (["33"], [1.0]),
    "38": (["35"], [1.0]),
    "34 - 37": (["34", "35", "36", "37"], [0.25, 0.25, 0.25, 0.25]),
    "39": (["39"], [1.0]),
    "40": (["40"], [1.0]),
    "41 / 43": (["41", "43"], [0.5, 0.5]),
    "48": (["45"], [1.0]),
    "44 - 47": (["44", "45", "46", "47"], [0.25, 0.25, 0.25, 0.25]),
    "49": (["49"], [1.0]),
    "32": (["34"], [1.0]),
    "50": (["50"], [1.0]),
    "52": (["54"], [1.0]),
    "56 - 58": (["56", "57", "58"], [1/3, 1/3, 1/3]),
    "60 - 61": (["60", "61"], [0.5, 0.5]),
    "62 - 67": (["62", "63", "64", "65", "66", "67"], [1/6, 1/6, 1/6, 1/6, 1/6, 1/6])
}


In [9]:
new_rows = []

for index, row in df.iterrows():
    old_acc_id = row['Acc-ID']
    
    # Check if the Acc-ID is a range and needs splitting
    if old_acc_id in hrm1_to_hrm2_dict:
        new_acc_ids, ratios = hrm1_to_hrm2_dict[old_acc_id]
        splits = len(new_acc_ids)

        for new_acc_id, ratio in zip(new_acc_ids, ratios):
            new_row = row.copy()
            new_row['Acc-ID'] = new_acc_id
            new_row['Budget y'] *= ratio
            new_row['Realized'] *= ratio
            new_row['Budget y+1'] *= ratio
            new_rows.append(new_row.to_dict())
    else:
        # Handle cases where Acc-ID is not in the mapping
        new_rows.append(row)

In [10]:
new_df = pd.DataFrame(new_rows, index=None)

In [13]:
new_df

,Acc-ID,Name,Budget y,Realized,Budget y+1
0,30,Personalaufwand,174335.600000,173668.702000,174267.000000
1,31,Sachaufwand,59494.900000,58819.409000,58983.700000
2,davon 314,Baulicher Unterhalt,6416.500000,6547.749000,6586.900000
3,34,Passivzinsen,2862.700000,1721.028000,2137.500000
4,330,Abschreibungen Finanzvermögen,3615.000000,3574.686000,3665.000000
5,331 - 333,Abschreibungen Verwaltungsvermögen,18425.700000,22119.987000,19325.900000
6,34,"Anteile, Entschädigungen, Beiträge",97236.700000,96152.367500,98641.250000
7,35,"Anteile, Entschädigungen, Beiträge",97236.700000,96152.367500,98641.250000
8,36,"Anteile, Entschädigungen, Beiträge",97236.700000,96152.367500,98641.250000
9,37,"Anteile, Entschädigungen, Beiträge",97236.700000,96152.367500,98641.250000


In [16]:
new_df["Acc-ID"] = new_df["Acc-ID"].astype(str)

# Keep only rows with numeric 'Acc-ID'
new_df = new_df[new_df["Acc-ID"].str.isnumeric()]
new_df = new_df[new_df['Acc-ID'] != '0']

/var/folders/vy/kftwr_sx3kjbxvnd1nvc851m0000gn/T/ipykernel_84177/3362964305.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Acc-ID"] = new_df["Acc-ID"].astype(str)


In [17]:
new_df

,Acc-ID,Name,Budget y,Realized,Budget y+1
0,30,Personalaufwand,174335.600000,173668.702000,174267.000000
1,31,Sachaufwand,59494.900000,58819.409000,58983.700000
3,34,Passivzinsen,2862.700000,1721.028000,2137.500000
4,330,Abschreibungen Finanzvermögen,3615.000000,3574.686000,3665.000000
6,34,"Anteile, Entschädigungen, Beiträge",97236.700000,96152.367500,98641.250000
7,35,"Anteile, Entschädigungen, Beiträge",97236.700000,96152.367500,98641.250000
8,36,"Anteile, Entschädigungen, Beiträge",97236.700000,96152.367500,98641.250000
9,37,"Anteile, Entschädigungen, Beiträge",97236.700000,96152.367500,98641.250000
16,35,Einlagen in Spezialfinanzierungen/Fonds,14949.000000,22073.590000,16501.000000
17,389,Einlagen in das Eigenkapital,0.000000,33100.000000,0.000000
